In [ ]:
# %load ResNet18.py

import tensorflow as tf
import cv2, numpy as np, os
import pandas as pd, time
import tensorflow.contrib as tf_contrib

from google.colab import drive
drive.mount("/content/drive")

decayOfBN = 0.0001
epsOfBN = 1e-05



def getMinibatches(feature, feature_label, crop = True, batch_size = 32):
    
    batch_n = feature.shape[0] // batch_size
    data = list(zip(feature, feature_label))
    np.random.shuffle(data)
    feature, feature_label = zip(*data)
    feature = np.array(feature)
    feature_label = np.array(feature_label)
    mini_batches = []
    i = 0
    for i in range(batch_n):
        feature_mini = feature[i * batch_size:(i + 1)*batch_size]
        if(crop):
            feature_mini = DataAugmentation(feature_mini, 224)
        feature_label_mini = feature_label[i * batch_size:(i + 1)*batch_size]
        mini_batches.append((feature_mini, feature_label_mini))
    if(feature.shape[0] % batch_size != 0):
        feature_mini = feature[i * batch_size:feature.shape[0]]
        if(crop):
            feature_mini = DataAugmentation(feature_mini, 224)
        feature_label_mini = feature_label[i * batch_size:feature_label.shape[0]]
        mini_batches.append((feature_mini, feature_label_mini))
        batch_n += 1
    return mini_batches, batch_n

def RandCrop(batch, crop_shape, padding=None):
    
    i_shape = np.shape(batch[0])
    if padding:
        i_shape = (i_shape[0] + 2 * padding, i_shape[1] + 2 * padding)
    new_batch = []
    npad = ((padding, padding), (padding, padding), (0, 0))
    for i in range(len(batch)):
        new_batch.append(batch[i])
        if padding:
            new_batch[i] = np.lib.pad(batch[i], pad_width=npad, mode='constant', constant_values=0)
        nh = np.random.randint(0, i_shape[0] - crop_shape[0])
        nw = np.random.randint(0, i_shape[1] - crop_shape[1])
        new_batch[i] = new_batch[i][nh:nh + crop_shape[0], nw:nw + crop_shape[1]]
    return new_batch
        
    
def DataAugmentation(batch, img_size):
    return RandCrop(batch, [img_size, img_size, 3])

def ReadImages(path, ran, phase):
    save_path = phase+"_images.npy"
    if(os.path.efeatureists(save_path)):
        return np.load(save_path)
    images = []
    for r, d, f in os.walk(path):
        for i in range(1, ran+1):
            img_name = os.path.join(r, str(i)+".jpg")
            images.append(cv2.imread(img_name)/255.)
    if(phase == 'test'):
        images = DataAugmentation(images, 224)
    print("done loading from " + path)
    images = np.array(images)
    np.save(save_path, images)
    return images

def ReadLabels(csv_file, phase, nb_classes = 8):
    
    labels = np.array(pd.read_csv(csv_file, header = None).values)
    labels = np.reshape(labels, (labels.shape[1], labels.shape[0]))
    one_hot_targets = np.eye(nb_classes)[[i-1 for i in labels]]
    one_hot_targets = np.reshape(one_hot_targets, (one_hot_targets.shape[0], 8))

    return one_hot_targets
    

def get_num_trainable_params():
    total_parameters = 0
    for variable in tf.trainable_variables():
        shape = variable.get_shape()
        variable_parameters = 1
        for dim in shape:
            variable_parameters *= dim.value
        total_parameters += variable_parameters
    print(total_parameters)





train_images = ReadImages("/content/drive/My Drive/Colab Notebooks/CVass2/train/", 1888, "train")
test_images = ReadImages("/content/drive/My Drive/Colab Notebooks/CVass2/test/", 800, "test")

train_labels = ReadLabels("/content/drive/My Drive/Colab Notebooks/CVass2/train_labels.csv", "train")
test_labels = ReadLabels("/content/drive/My Drive/Colab Notebooks/CVass2/test_labels.csv", "test")





def resBlock(feature, filter_size, wt_init, weight_regularizer, conv_number, instance, is_train, stride = 1, reuse = False):
    prev = feature
      
    feature = tf.layers.conv2d(feature, filters=filter_size, kernel_size=3, kernel_initializer=wt_init, strides=stride,
                                kernel_regularizer=weight_regularizer, padding="SAME", trainable=is_train, name="shortcut_conv"+conv_number+"_"+instance, reuse=reuse)
    feature = tf_contrib.layers.batch_norm(feature, decay=decayOfBN, epsilon=epsOfBN, center=True, scale=True, trainable=is_train, reuse=reuse, scope="short_batch_norm"+conv_number+"_"+instance)
    feature = tf.nn.relu(feature)
    feature = tf.layers.conv2d(feature, filters=filter_size, kernel_size=3, kernel_initializer=wt_init, trainable=is_train,
                         kernel_regularizer=weight_regularizer, name="conv"+conv_number+"_"+instance, padding="SAME", reuse=reuse)
    
    feature = tf_contrib.layers.batch_norm(feature, decay=decayOfBN, epsilon=epsOfBN, center=True, scale=True, trainable=is_train, reuse=reuse, scope="batch_norm"+conv_number+"_"+instance)
    
    if(stride == 2):
        prev = tf.layers.conv2d(prev, filters=filter_size, kernel_size=1, kernel_initializer=wt_init, strides=stride,
                                kernel_regularizer=weight_regularizer, padding="SAME", trainable=is_train, name="1feature1_"+conv_number+"_"+instance, reuse=reuse)
    
    feature = prev + feature
    feature = tf.nn.relu(feature)
    
    return feature

def BuildModel(train_input, is_train = False, reuse = False):
    wt_init = tf_contrib.layers.featureavier_initializer()
    weight_regularizer = tf_contrib.layers.l2_regularizer(0.0001)
    
    feature = tf.layers.conv2d(train_input, filters=64, kernel_size=7, kernel_initializer=wt_init, strides=2, 
                         kernel_regularizer=weight_regularizer, trainable=is_train, name="conv1", padding='SAME', reuse=reuse)
    feature = tf_contrib.layers.batch_norm(feature, decay=decayOfBN, epsilon=epsOfBN, center=True, scale=True, trainable=is_train, reuse=reuse, scope="batch_norm1")
    feature = tf.nn.relu(feature)
    
    feature = tf.layers.mafeature_pooling2d(feature, pool_size = 2, strides = 2, name="mafeaturepool") # 56feature56feature64
    
    feature = resBlock(feature, 64, wt_init, weight_regularizer, "2", "1", is_train, reuse=reuse)
    feature = resBlock(feature, 64, wt_init, weight_regularizer, "2", "2", is_train, reuse=reuse)
    
    feature = resBlock(feature, 128, wt_init, weight_regularizer, "3", "1", is_train, stride=2, reuse=reuse)
    feature = resBlock(feature, 128, wt_init, weight_regularizer, "3", "2", is_train, reuse=reuse)
    
    feature = resBlock(feature, 256, wt_init, weight_regularizer, "4", "1", is_train, stride=2, reuse=reuse)
    feature = resBlock(feature, 256, wt_init, weight_regularizer, "4", "2", is_train, reuse=reuse)
    
    feature = resBlock(feature, 512, wt_init, weight_regularizer, "5", "1", is_train, stride=2, reuse=reuse)
    feature = resBlock(feature, 512, wt_init, weight_regularizer, "5", "2", is_train, reuse=reuse)
    
    feature = tf.layers.average_pooling2d(feature, pool_size=7, strides=2, name="avgpool")
    feature = tf.contrib.layers.flatten(feature)
     
    feature = tf.layers.dense(feature, 8, kernel_initializer=wt_init, trainable=is_train, name="output", reuse=reuse)  
    return feature

def GetLosses(output, label):
    loss = tf.reduce_mean(tf.nn.softmafeature_cross_entropy_with_logits_v2(logits=output, labels=label))
    prediction = tf.equal(tf.argmafeature(output, -1), tf.argmafeature(label, -1))
    accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))
    
    return loss, accuracy





tf.reset_default_graph()
train_input = tf.placeholder(tf.float32, (None, 224, 224, 3), name = "train_input")
train_label = tf.placeholder(tf.int32, (None, 8), name = "train_label")
test_input = tf.placeholder(tf.float32, (None, 224, 224, 3), name = "test_input")
test_label = tf.placeholder(tf.int32, (None, 8), name = "test_label")

batch = 32

lr = tf.placeholder(tf.float32, name='learning_rate')
train_output = BuildModel(train_input, is_train = True)
test_output = BuildModel(test_input, reuse = True)

train_loss, train_acc = GetLosses(train_output, train_label)
test_loss, test_acc = GetLosses(test_output, test_label)

optimizer = tf.train.MomentumOptimizer(learning_rate=lr, momentum=0.9).minimize(train_loss)


summary_train_loss = tf.summary.scalar("train_loss", train_loss)
summary_test_loss = tf.summary.scalar("train_loss", test_loss)

summary_train_accuracy = tf.summary.scalar("train_accuracy", train_acc)
summary_test_accuracy = tf.summary.scalar("train_accuracy", test_acc)

train_summary = tf.summary.merge([summary_train_loss, summary_train_accuracy])
test_summary = tf.summary.merge([summary_test_loss, summary_test_accuracy])


epo = 40
epo_n = 0.1
cntr = 0
test_cntr = 0

sess = tf.Session()
sess.run(tf.global_variables_initializer())

checkpoint_dir = "./checkpoints"
writer = tf.summary.FileWriter("./logs", sess.graph)
saver = tf.train.Saver()

print("Starting Train!!!")
start_time = time.time()

test_accs = []
test_losses = []
for each_epo in range(epo):
    if each_epo == int(epo * 0.5) or each_epo == int(epo * 0.75):
        epo_n = epo_n * 0.1
    each_minibatches, n_batches = getMinibatches(train_images, train_labels, batch_size = batch)
    i = 0
    for each_minibatch in each_minibatches:
        i += 1
        feature, y = each_minibatch
        
        train_feed_dict = {train_input:feature, train_label: y, lr: epo_n}
        
        acc, _, l, summary1 = sess.run([train_acc, optimizer, train_loss, train_summary], feed_dict=train_feed_dict)
        writer.add_summary(summary1, cntr)
        cntr += 1
        
        print("each_epo: [%2d] [%5d/%5d], train_accuracy: %.2f, learning_rate : %.4f, train_loss: %.2f"                       % (each_epo, i, n_batches, acc, epo_n, l))
    if(each_epo%5 == 0):
        if not os.path.efeatureists(checkpoint_dir):
            os.makedirs(checkpoint_dir)
        saver.save(sess, os.path.join(checkpoint_dir, 'ResNet18.model'), global_step=cntr)
        
    each_minibatches, n_batches = getMinibatches(test_images, test_labels, batch_size = batch, crop=False)
    for each_minibatch in each_minibatches:
        feature, y = each_minibatch
        
        test_feed_dict = {test_input:feature, test_label: y}
        
        t_acc, summary2, t_loss = sess.run([test_acc, test_summary, test_loss], feed_dict=test_feed_dict)
        
        test_accs.append(t_acc)
        test_losses.append(t_loss)
        writer.add_summary(summary2, test_cntr)
        print("each_epo: [%2d], test_accuracy: %.2f, test_loss: %.2f"%(each_epo, t_acc, t_loss))
        test_cntr += 1
    
sess.close()
